## Model evaluation
---
### Import Statements 

In [ ]:
import os 
import sys
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np
from numpy import load
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.colors import ListedColormap
from statistics import mean
from skimage.measure import label
from scipy.ndimage import binary_dilation, binary_closing
import pandas as pd
from skimage.util import montage
from scipy.ndimage import zoom
import import_ipynb
from img_functions_dicom_nsclc_interobserver import *
import tensorflow as tf

### Settings

In [ ]:
prediction_1_path = "" #path to high res, low input volume model predictions (generated from predict script)
prediction_2_path = "" #path to low res, high input volume model predictions (generated from predict script)
lungseg_prediction_path = "" #path to lung segmentation model predictions (generated from predict script)


mask_path = "" #path to ground truth masks
ct_path = "" #path to CTs

ids = next(os.walk(prediction_1_path))[2]
print(ids)

In [ ]:
ct_level_results = {'id': [],
                   'prob max': [],
                   'dice': [],
                   'FP lesions': [],
                   'FN lesions': [],
                   'total_predicted_voxels': [],
                   'total_actual_voxels': []}

lesion_level_results = {'id':[],
                        'lesion dice': [],
                        'lesion voxels': [],
                       'predicted lesion voxels': []}

### Evaluate

In [ ]:
for _id in ids:
    
    mask = load("{mask_path}{_id}".format(mask_path=mask_path, _id=_id))['arr_0']
    
    prediction_1 = load("{prediction_path}{_id}".format(prediction_path=prediction_1_path, _id=_id))['arr_0']
    prediction_2 = load("{prediction_path}{_id}".format(prediction_path=prediction_2_path, _id=_id))['arr_0']
    lungseg_prediction = load("{prediction_path}{_id}".format(prediction_path=lungseg_prediction_path, _id=_id))['arr_0']

    ct = load("{ct_path}{_id}".format(ct_path=ct_path, _id=_id))['arr_0']
    
    # Reshape to match sizes
    prediction_1 = zoom(prediction_1, tuple([i/j for i,j in zip(prediction_2.shape,prediction_1.shape)]))
    
    prediction_average = np.mean([prediction_1, prediction_2], axis=0)
    
    lungseg_prediction[lungseg_prediction<=0.5] = 0
    lungseg_prediction[lungseg_prediction>0.5] = 1
    
    prediction_thresholding = np.copy(prediction_average)
    prediction_thresholding[prediction_thresholding<=0.45] = 0
    prediction_thresholding[prediction_thresholding>0.45] = 1
    prediction = prediction_thresholding
    
    # Remove any tumor predictions that are 600 cm below top of CT
    if prediction.shape[2]*1.5*2.08 > 600:
        lb = round(prediction.shape[2]-(600//(1.5*2.08)))
        prediction[:,:,:lb] = 0
        lungseg_prediction[:,:,:lb] = 0

    # Remove any tumor predictions that fall outside lung segmentation, and any that are less than 65 mm^3 (diameter of 5 mm)
    lungseg_prediction_dilation = binary_dilation(lungseg_prediction, iterations=5)
    conn_components = label(prediction, connectivity=2, return_num=True)
    conn_arr = conn_components[0]
    prediction_conn_arr = np.copy(conn_arr)
    conn_labels = conn_components[1]
    if conn_arr[0,0,0] == conn_arr[0,conn_arr.shape[1]-1,0] == conn_arr[0,0,conn_arr.shape[2]-1]:
        corner_label = conn_arr[0,0,0]
    else:
        raise Exception("corner pixels are different")
    removed_predictions = []
    for i in range(conn_labels+1):
        if i == corner_label:
            conn_arr[conn_arr==i] = 0
        elif ((conn_arr==i)*(lungseg_prediction_dilation)).max() == False: # if none of the connected component filters fall within dilated lung segmentation
                conn_arr[conn_arr==i] = 0
                removed_predictions.append(i)
    
        elif (conn_arr == i).sum() < 65/((0.9765625*2.08)*(0.9765625*2.08)*(1.5*2.08)): # CHANGE THIS TO BE DYNAMIC BASED ON VOXEL DIMENSIONS
            conn_arr[conn_arr==i] = 0
            removed_predictions.append(i)
    
    prediction_conn_arr = np.copy(conn_arr)
    conn_arr = np.where(conn_arr != 0 , 1, conn_arr)
    prediction = conn_arr
    
    # Lesion level statistics
    mask_components = label(mask, connectivity=2, return_num=True)
    mask_conn_arr = mask_components[0]
    mask_labels = mask_components[1]
    
    if mask_conn_arr[0,0,0] == mask_conn_arr[0,mask_conn_arr.shape[1]-1,0] == mask_conn_arr[0,0,mask_conn_arr.shape[2]-1]:
        mask_corner_label = mask_conn_arr[0,0,0]
    else:
        raise Exception("corner pixels are different")
    
    false_negative_lesions = 0
    
    prediction_labels = conn_labels
    false_positive_lesions = [i for i in list(range(prediction_labels+1)) if i not in [corner_label]] # Start with all predicted tumors being considered as false positive, then remove as you find components that overlap with any ground truth tumor
    false_positive_lesions = [i for i in false_positive_lesions if i not in removed_predictions] # remove any of the predicted tumors that did not fall in lungs
    
    
    lesion_dices = []
    for i in range(mask_labels+1):
        if i != mask_corner_label and (mask_conn_arr == i).sum() > 65/((0.9765625*2.08)*(0.9765625*2.08)*(1.5*2.08)): 
            
            lesion_mask_arr = np.copy(mask_conn_arr)
            lesion_mask_arr = np.where(lesion_mask_arr == i, 1, 0)
            
            overlapping_prediction_components = []
            
            for j in range(prediction_labels+1):

                if j != corner_label: # corner_label = prediction array corner label
                    if ((prediction_conn_arr==j)*(lesion_mask_arr.astype(bool))).max() == True: # if any pixels of the connected component falls within the ground truth tumor
                        overlapping_prediction_components.append(j)

            #print("overlapping prediction components:", overlapping_prediction_components)
            false_positive_lesions = [i for i in false_positive_lesions if i not in overlapping_prediction_components] # remove any predicted tumors that overlap with ground truth tumor (b/c these are not false positive)
            
            if len(overlapping_prediction_components) == 0: # add to false negative lesion if current ground truth lesion has not tumor predictions that fall within it 
                false_negative_lesions += 1

            lesion_prediction_arr = np.zeros(prediction_conn_arr.shape)
            
            for k in overlapping_prediction_components:
                lesion_prediction_arr[prediction_conn_arr==k] = 1

            lesion_dice = (2*(lesion_prediction_arr*lesion_mask_arr).sum())/(lesion_prediction_arr.sum() + lesion_mask_arr.sum())
            
            # Save lesion level results
            lesion_level_results['id'].append(_id)
            lesion_level_results['lesion dice'].append(lesion_dice)
            lesion_level_results['lesion voxels'].append(lesion_mask_arr.sum())
            lesion_level_results['predicted lesion voxels'].append(lesion_prediction_arr.sum())
            lesion_dices.append(lesion_dice)
        
        elif (mask_conn_arr == i).sum() < 65/((0.9765625*2.08)*(0.9765625*2.08)*(1.5*2.08)):
            mask_conn_arr[mask_conn_arr==i] = 0
        
    mask_conn_arr[mask_conn_arr!=0] = 1
    
    mask = mask_conn_arr
    
    # Dice
    dice = (2*(prediction*mask).sum())/(prediction.sum() + mask.sum())
    print("{}: {}".format(_id,dice))
    
    # Save ct level results
    ct_level_results['id'].append(_id)
    ct_level_results['prob max'].append(prediction.max())
    ct_level_results['dice'].append(dice)
    ct_level_results['FP lesions'].append(len(false_positive_lesions))
    ct_level_results['FN lesions'].append(false_negative_lesions)